In [14]:
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio
import pandas as pd
import numpy as np
import torch
from bc import bit_cypher
import os
import random

In [29]:
files = os.walk('../data/train_files_48k')
sr = 48000

In [16]:
files = []
for root,dirs,file in os.walk('../data/train_files_48k'):
    for f in file:
        path = os.path.join(root,f)
        files.append(path)



In [27]:
sample = random.sample(files,1000)
train_files = sample[0:800]
test_files = sample[800:]

In [39]:
x = torch.stack([torch.tensor(librosa.load(x,sr=sr)[0]) for x in train_files])

In [41]:
print('num unique before quantize')
pd.Series(x.flatten()).nunique()

96764657